In [1]:
from docplex.mp.model import Model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Clase instancia

class Instancia:
    def __init__(self, instance = "instancias/n20w120.001.txt"):
        self.instance_name = instance.split("/")[1]
        print(self.instance_name)
        # Parámetros
        self.M = 10000
        self.cust_no = None
        self.coord_x = None
        self.coord_y = None
        self.demand = None
        self.ready_time = None
        self.due_date = None
        self.service_time = None
        # Leer archivo
        self.problem = self.leer_archivo(instance)
    
    def leer_archivo(self,instance):
        # Lectura de instancias
        file_head = []
        cols_head = []
        instance_data = []
        with open(instance, "r") as f:
            file = f.readlines()
            file_head = file[0].replace("\n","").split("\t")[0].split(" ")
            file_head = "{} {} {}".format(file_head[1],file_head[5],file_head[6])
            cols_head = file[3].replace("\n","").split("\t")
            for line in file[6:-1]:
                splited_line = line.replace("\n","").split("\t")
                instance_data.append(splited_line)

        # Convertir en array
        instance_data = np.array(instance_data).astype(int)
        self.cust_no = instance_data[:,0]
        self.coord_x = instance_data[:,1]
        self.coord_y = instance_data[:,2]
        self.demand = instance_data[:,3]
        self.ready_time = instance_data[:,4]
        self.due_date = instance_data[:,5]
        self.service_time = instance_data[:,6]
        self.file_head = file_head
        
        
        self.V = instance_data[:,0]
        self.arcos = [(i,j) for i in self.V for j in self.V if i!=j]
        self.a = {i: instance_data[:,4][i-1] for i in self.V}
        self.b = {i: instance_data[:,5][i-1] for i in self.V}
        self.c = {(i,j): int(np.hypot(self.coord_x[i-1] - self.coord_x[j-1] , self.coord_y[i-1] - self.coord_y[j-1])) for i,j in self.arcos}
        # self.W = {i: (self.a[i], self.b[i]) for i in self.V}
        self.W = {i: [i for i in range(self.a[i], self.b[i])] for i in self.V}
        self.p = self.V[0]
        self.q = self.V[-1]

        # self.I = {}
        

    def __repr__(self):
        return("Instancia: {} con {} clientes y header {}".format(self.instance_name,len(self.demand), self.file_head))

class ModeloClasico():
    def __init__(self):
        self.name = "TSPTW"

    def build(self,ins):
        # Modelo
        self.mdl = Model('TSPTW')

        # Variables de decisión
        self.x = self.mdl.binary_var_dict(ins.arcos,name= 'x')
        self.u = self.mdl.integer_var_dict(ins.V, name= 'u', lb=0)

        # Restricción 1: Funcion Objetivo
        self.mdl.minimize(self.mdl.sum(self.x[(i,j)]*ins.c[(i,j)] for i,j in ins.arcos))

        # Restricción 2: Un arco saliente de cada vertice
        for i in ins.V:
            self.mdl.add_constraint(self.mdl.sum(self.x[(i,j)] for j in ins.V if i!=j) == 1, ctname = "restriccion2: x_%d" %(i))

        # Restricción 3: Un arco saliente de cada vertice
        for i in ins.V:
            self.mdl.add_constraint(self.mdl.sum(self.x[(j,i)] for j in ins.V if i!=j) == 1, ctname = "restriccion3: x_%d" %(i))

        # Restricción 4: Subrutas MTZ
        for i,j in ins.arcos:
            if i!=1:
                self.mdl.add_constraint(self.u[i]-self.u[j] + ins.M*self.x[(i,j)] <= ins.M - ins.c[(i,j)], ctname = "restriccion4: x_%d_%d" %(i,j))

        # Restricción 5: Ventanas de Tiempo
        for i in ins.V:
            self.mdl.add_constraints([ins.a[i] <= self.u[i] ,self.u[i] <= ins.b[i]])

        # print(self.mdl.pprint_as_string())

    def solve(self):
        self.solucion = self.mdl.solve(log_output= True)

        print(self.mdl.get_solve_status())
        print(self.solucion)
        self.arcos_solucion = [i for i in ins.arcos if self.x[i].solution_value>0.9]
        print(self.arcos_solucion)



    def plot(self,ins):
        fig, ax = plt.subplots(figsize=(12,7))
        for i in range(len(ins.V)):
            if i!=0:
                ax.scatter(ins.coord_x[i],ins.coord_y[i],300,color="black",marker = 's',zorder=2)
                ax.annotate(str(i +1),xy=(ins.coord_x[i],ins.coord_y[i]),xytext=(ins.coord_x[i]-0.5,ins.coord_y[i]-0.5), color="white")
            else:
                ax.scatter(ins.coord_x[i],ins.coord_y[i],300,color="red",marker = 's',label='Depósito',zorder=2)
                ax.annotate(str(i + 1),xy=(ins.coord_x[i],ins.coord_y[i]),xytext=(ins.coord_x[i]-0.5,ins.coord_y[i]-0.5), color="white")

        for i,j in self.arcos_solucion:
            plt.plot([ins.coord_x[i-1],ins.coord_x[j-1]],[ins.coord_y[i-1],ins.coord_y[j-1]],color='black',zorder=1)

        DPatch = mpatches.Patch(color='red',label='Deposito')
        ISolPatch = mpatches.Patch(color='black',label='Cliente')
        plt.legend(handles=[DPatch, ISolPatch])

        plt.xlabel("Coordenada x")
        plt.ylabel("Coordenada y")
        plt.title("Solución instancia")
        
        # plt.savefig("fig.png")
        plt.show()


    def get_permutacion(self,arcos):
        camino = []
        n = len(arcos)
        actual = arcos[0][0]
        sig = arcos[0][1]
        camino.append(actual)
        camino.append(sig)
        ultimo = int()
        i = 1

        while camino[0] != ultimo:
            if sig == arcos[i][0]:
                actual = sig
                sig = arcos[i][1]
                if sig not in camino:
                    camino.append(sig)
                ultimo = sig
                i = 1
            else:
                i +=1
        print(camino)

        return(camino)

class ModeloHeuristico():
    def __init__(self):
        self.name=None

    def random_solution(n_nodos = 21, semilla=2):
        np.random.seed(semilla)
        l = [i for i in range(1, n_nodos + 1)]
        np.random.shuffle(l)
        return(l)
    # Initial: validar, better, local1shift, permutacion, random

    def validar_solucion(camino):
        print(camino)
        u = {}
        u[camino[0]] = 0
        # print(a)
        # print(b)
        # print(c)
        for i in range(len(camino)):
            try:
                # print(i, camino[i], camino[i+1])
                
                # u[camino[i+1]] = u[camino[i]] + c[(camino[i], camino[i + 1])]

                # print(u[camino[i]] + c[(camino[i], camino[i + 1])] <= a[camino[i + 1]]) # => i+1 esta cerrado a la hora que llega => u[camino[i+1]] = a[camino[i + 1]]

                if u[camino[i]] + c[(camino[i], camino[i + 1])] >= a[camino[i + 1]]:
                    if u[camino[i]] + c[(camino[i], camino[i + 1])] <= b[camino[i + 1]]:
                        u[camino[i+1]] = u[camino[i]] + c[(camino[i], camino[i + 1])]
                    else:
                        return(False)
                else:
                    u[camino[i+1]] = a[camino[i + 1]]
            
            except:
                if u[camino[i]] + c[(camino[i], camino[0])] >= a[camino[0]]:
                    if u[camino[i]] + c[(camino[i], camino[0])] <= b[camino[0]]:
                        u[camino[0]] = u[camino[i]] + c[(camino[i], camino[0])]
                    else:
                        return(False)
                else:
                    u[camino[0]] = a[camino[0]]

        return(True)

    def DosOpt(ciudad,ins):
        n = len(ciudad)
        flag = True
        contar = 0
        actual = 0
        k = np.random.randint(0, n - 1)
        ciudad = ciudad[k:] + ciudad[:k] # sin numpy

        for i in range(n - 2):
            for j in range(i + 1, n - 1):
                nuevoCosto = c[ciudad[i], ciudad[j]] + c[(ciudad[i + 1], ciudad[j + 1])] - c[(ciudad[i], ciudad[i + 1])] - c[(ciudad[j], ciudad[j + 1])]
                if nuevoCosto < actual:
                    if ModeloHeuristico.validar_solucion(ciudad):
                        actual = nuevoCosto
                        min_i, min_j = i, j
                        # Al primer cambio se sale
                        contar += 1
                        if contar == 1 :
                            flag = False

            if flag == False:
                break
        # Actualiza la subruta se encontró
        if contar < 0:
            ciudad[min_i + 1 : min_j + 1] = ciudad[min_i + 1 : min_j + 1][::-1]

    # pi_optimo = modelo.get_permutacion(modelo.arcos_solucion)

    camino = random_solution(21)
    # validar_solucion(pi_optimo)


    ciudades = [i for i in range(1,20)]
    ventanas = {}
    for i in range(len(ciudades)):
        #ventanas[ciudades[i]] = ins.a[i], ins.b[i]
        pass



    def Local1Shift(ciudades, ventanas, distancias): #Ciudades es el conjunto de ciudades en orden del camino y ventanas es un diccionario con ciudad :(a,b)
        def fueraDeVentana(ciudades, ventanas, distancias): #tiene que devolver los nodos y sus pos, en el caso de que no cumplan sus ventanas
            u = 0 # tiempo acumulado en la ciudad i
            flag = False
            infactibles = {}
            for i in range(len(ciudades)-1,-1,-1): 
                if i == 0:
                    u += distancias[(ciudades[len(ciudades)-1],ciudades[i])]
                    if u >= ventanas[ciudades[i]][0] and u <= ventanas[ciudades[i]][1]:
                        infactibles[ciudades[i]] = [False, u]
                    else:
                        infactibles[ciudades[i]] = [True, u]
                    
                else: #i != 0 and i != len(ciudades)
                    u += distancias[(ciudades[i-1],ciudades[i])]
                    if u >= ventanas[ciudades[i]][0] and u <= ventanas[ciudades[i]][1]:
                        infactibles[ciudades[i]] = [False, u]
                    else:
                        infactibles[ciudades[i]] = [True, u]

            return infactibles #False == factible, True == infactible

        def verificar(ciudades, ventanas, distancias,p): #p=1 si se queire obtener la pos del primer infactible #p=0 si se queire obtener la pos del primer factible 
            infactibles = fueraDeVentana(ciudades, ventanas, distancias)

            if p == 1:
                for i in infactibles:
                    if infactibles[i][0] == True:
                        pos = ciudades.index(i)
                        flag = True #Con que solo una ciudad no cumpla, el camino es infactible
                        break #Se hace solo para el primer infactible
                    else: #No hay infactibles
                        pos = None
                        flag = False
                return infactibles, pos, flag
            else: #p==0
                for i in infactibles:
                    if infactibles[i][0] == True:
                        pos = ciudades.index(i)
                        flag = True #Con que solo una ciudad no cumpla, el camino es infactible
                    else:
                        pos = None
                        flag = False
                        break #Se hace solo para el primer infactible
                
                return infactibles, pos, flag


        def backwardViolated(ciudades, ventanas, distancias):
            infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
            if pos == 0:
                aux = ciudades[pos]
                ciudades[pos] = ciudades[len(ciudades)-1]
                ciudades[len(ciudades)-1] = aux
            
            if pos == None:
                pass

            else:
                for i in range(pos,-1,-1):
                    if i != pos:
                        if infactibles[ciudades[pos]][1] + distancias[(ciudades[i],ciudades[pos])] <= ventanas[ciudades[pos]][1]:
                            pos2 = i
                            break #El mas cercano al infactible que cumpla con el criterio de ventana
                        else:
                            pos2 = None
                if pos2 != None:
                    aux = ciudades[pos]
                    for i in range(pos,pos2,-1): #Se corren +1 las posiciones de las ciudades, queda la ciudad en la pos2 repetida
                        ciudades[i] = ciudades[i-1]
                    ciudades[pos2] = aux

            return ciudades

        def forwardNonViolated(ciudades, ventanas, distancias):
            infactibles, pos, flag = verificar(ciudades, ventanas, distancias,0)
            if pos == len(ciudades)-1:
                aux = ciudades[pos]
                ciudades[pos] = ciudades[0]
                ciudades[0] = aux

            if pos == None:
                pass

            else:
                for i in range(pos+1,len(ciudades)):
                    if i != pos:
                        if infactibles[ciudades[pos]][1] + distancias[(ciudades[i],ciudades[pos])] >= ventanas[ciudades[pos]][1]: #Queda fuera de la ventana
                            pos2 = i
                            break #El mas cercano al infactible que cumpla con el criterio de ventana
                        else:
                            pos2 = None
                if pos2 != None:
                    aux = ciudades[pos]
                    #for i in range(pos,pos2,-1): #Se corren -1 las posiciones de las ciudades, queda la ciudad en la pos2 repetida
                    for i in range(pos,pos2):
                            ciudades[i] = ciudades[i+1]
                    ciudades[pos2] = aux

            return ciudades
        
        def forwardViolated(ciudades, ventanas, distancias):
            infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
            if pos == len(ciudades)-1:
                aux = ciudades[pos]
                ciudades[pos] = ciudades[0]
                ciudades[0] = aux
            if pos == None:
                pass
            else:    
                for i in range(pos+1,len(ciudades)):
                    if i != pos:
                        if infactibles[ciudades[pos]][1] + distancias[(ciudades[i],ciudades[pos])] >= ventanas[ciudades[pos]][1]: #Queda fuera de la ventana
                            pos2 = i
                            break #El mas cercano al infactible que cumpla con el criterio de ventana
                        else:
                            pos2 = None
                if pos2 != None:
                    aux = ciudades[pos]
                    #for i in range(pos,pos2,-1): #Se corren -1 las posiciones de las ciudades, queda la ciudad en la pos2 repetida
                    for i in range(pos,pos2):
                            ciudades[i] = ciudades[i+1]
                    ciudades[pos2] = aux

            return ciudades
        
        def backwardNonViolated(ciudades, ventanas, distancias):
            infactibles, pos, flag = verificar(ciudades, ventanas, distancias,0)
            if pos == 0:
                aux = ciudades[pos]
                ciudades[pos] = ciudades[len(ciudades)-1]
                ciudades[len(ciudades)-1] = aux
            if pos == None:
                pass
            else:
                for i in range(pos,-1,-1):
                    if i != pos:
                        if infactibles[ciudades[pos]][1] + distancias[(ciudades[i],ciudades[pos])] <= ventanas[ciudades[pos]][1]:
                            pos2 = i
                            break #El mas cercano al infactible que cumpla con el criterio de ventana
                        else:
                            pos2 = None
                if pos2 != None:
                    aux = ciudades[pos]
                    for i in range(pos,pos2,-1): #Se corren +1 las posiciones de las ciudades, queda la ciudad en la pos2 repetida
                        ciudades[i] = ciudades[i-1]
                    ciudades[pos2] = aux

            return ciudades
                
        #Main           
        it = 0 #nr de iteraciones max
        #it_max = len(ciudades)*len(ciudades)
        it_max = 1000

        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)

        print('ciudades ini', ciudades)

        while flag == True :
            ciudades = backwardViolated(ciudades, ventanas, distancias)
            infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
            ciudades = forwardNonViolated(ciudades, ventanas, distancias)
            infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
            ciudades = backwardNonViolated(ciudades, ventanas, distancias)
            infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
            ciudades = forwardViolated(ciudades, ventanas, distancias)
            infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
            it += 1
            if it == it_max + 100:
                print('No hay factibilidad')
                break
            #print('it',it)
        print(ciudades)


    # ciudades = Local1Shift(ciudades, ventanas, ins.c)


class ModeloTimeIndexed():
    def __init__(self):
        self.name = "TSPTW-TIME INDEXED"

    def build(self,ins):
        # Modelo
        self.mdl = Model('TSPTW - TIME INDEXED')

        # Variables de decisión
        self.x = self.mdl.binary_var_dict( {(i,j) for i in ins.V for j in ins.V}, name = 'x')
        self.y = self.mdl.binary_var_dict( {(i,j,t) for i in ins.V for j in ins.V for t in ins.W[i]}, name = 'y')
        self.z = self.mdl.binary_var_dict( {(i,t) for i in ins.V for t in ins.W[i]}, name = 'z')
        
        # Restricción 1: Funcion Objetivo
        self.mdl.minimize(self.mdl.sum(self.x[(i,j)]*ins.c[(i,j)] for i,j in ins.arcos))

        # Restricción 2: La ruta comienza en cada nodo i dentro de la ventana de tiempo
        for i in ins.V:
            self.mdl.add_constraint(self.mdl.sum(self.z[(i,t)] for t in ins.W[i]) == 1 , ctname= 'restriccion2: z_{}'.format(i))

        # Restricción 3: 
        for i in ins.V:
            if i!=ins.q:
                for t in ins.W[i]:
                    self.mdl.add_constraint(self.mdl.sum(self.y[i,j,t] for j in ins.V) == self.z[i,t], ctname= 'restriccion3: z_{}'.format(i))
        # Restriccion 4:
        for i in ins.V:
            if i!=ins.p:
                for t in ins.W[i]:
                    self.mdl.add_constraint(self.mdl.sum(  self.mdl.sum(self.y[k,i,tau] for tau in [tau for tau in ins.W[k] if tau <= ins.a[i] - ins.c[k,i]]) for k in ins.V if k!=i ) == self.z[i,t], ctname= 'restriccion4: z_{}'.format(i))

        # Restriccion 5:
        for i,j in ins.arcos:
            self.mdl.add_constraint(self.mdl.sum(self.y[i,j,t] for t in ins.W[i]) == self.x[i,j], ctname= 'restriccion5: y_{}_{}_{}'.format(i,j,t))

    def solve(self):
        self.solucion = self.mdl.solve(log_output= True)
        print(self.mdl.get_solve_status())
        print(self.solucion)
        self.arcos_solucion = [i for i in ins.arcos if self.x[i].solution_value>0.9]
        print(self.arcos_solucion)



    def plot(self,ins):
        fig, ax = plt.subplots(figsize=(12,7))
        for i in range(len(ins.V)):
            if i!=0:
                ax.scatter(ins.coord_x[i],ins.coord_y[i],300,color="black",marker = 's',zorder=2)
                ax.annotate(str(i +1),xy=(ins.coord_x[i],ins.coord_y[i]),xytext=(ins.coord_x[i]-0.5,ins.coord_y[i]-0.5), color="white")
            else:
                ax.scatter(ins.coord_x[i],ins.coord_y[i],300,color="red",marker = 's',label='Depósito',zorder=2)
                ax.annotate(str(i + 1),xy=(ins.coord_x[i],ins.coord_y[i]),xytext=(ins.coord_x[i]-0.5,ins.coord_y[i]-0.5), color="white")

        for i,j in self.arcos_solucion:
            plt.plot([ins.coord_x[i-1],ins.coord_x[j-1]],[ins.coord_y[i-1],ins.coord_y[j-1]],color='black',zorder=1)

        DPatch = mpatches.Patch(color='red',label='Deposito')
        ISolPatch = mpatches.Patch(color='black',label='Cliente')
        plt.legend(handles=[DPatch, ISolPatch])

        plt.xlabel("Coordenada x")
        plt.ylabel("Coordenada y")
        plt.title("Solución instancia")
        
        # plt.savefig("fig.png")
        plt.show()


    def get_permutacion(self,arcos):
        camino = []
        n = len(arcos)
        actual = arcos[0][0]
        sig = arcos[0][1]
        camino.append(actual)
        camino.append(sig)
        ultimo = int()
        i = 1

        while camino[0] != ultimo:
            if sig == arcos[i][0]:
                actual = sig
                sig = arcos[i][1]
                if sig not in camino:
                    camino.append(sig)
                ultimo = sig
                i = 1
            else:
                i +=1
        print(camino)

        return(camino)

modelo = ModeloTimeIndexed()
modelo.build(ins)
print(modelo.mdl.export_to_string())


NameError: name 'ins' is not defined

In [3]:
ins = Instancia()
modelo = ModeloClasico()
modelo.build(ins)
modelo.solve()
# modelo.plot(ins)

n20w120.001.txt
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 242 rows and 105 columns.
MIP Presolve modified 400 coefficients.
Reduced MIP has 242 rows, 336 columns, and 1232 nonzeros.
Reduced MIP has 316 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (1.01 ticks)
Probing fixed 17 vars, tightened 0 bounds.
Probing time = 0.04 sec. (0.88 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 0 rows and 17 columns.
MIP Presolve modified 4 coefficients.
Reduced MIP has 242 rows, 319 columns, and 1198 nonzeros.
Reduced MIP has 299 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.94 ticks)
Probing time = 0.02 sec. (0.81 ticks)
Clique table members: 304.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxatio

In [5]:
modelo.mdl.solucion()

AttributeError: 'Model' object has no attribute 'solucion'

In [ ]:
modelo.solve()